FileName = BidirLSTM_GRU_Glove.ipynb

The code in the notebook does classification of the text from IMDB. It does not do Text Gen

1. Data comes from IMDB
2. There are 2 models in notebook:
   1. LSTM Bi-directional,
   2. GRU

Note that the last model uses pre-trained Embessings (Glove)  



In [1]:
import re
import string
import sys

from logging import logProcesses
import os, pathlib, shutil, random
from platform import python_branch
from syslog import LOG_SYSLOG

import numpy as np
# 
# import matplotlib.pyplot as plt 
# 
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import optimizers

2025-03-28 16:31:08.971596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743193869.438228   33122 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743193869.549286   33122 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 16:31:10.677704: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
print("Python Version:", sys.version)
print ("TF Version   ", tf.__version__)
print ("TF Path      ", tf.__path__[0])
print("Keras version ", keras.__version__)
print("numpy version ", np.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print("GPUS =", gpus)

Python Version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
TF Version    2.19.0
TF Path       /drv3/hm3/code/python/tf2.19/lib/python3.12/site-packages/keras/api/_v2
Keras version  3.9.0
numpy version  2.1.3
Num GPUs Available:  1
GPUS = [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
batch_size = 12
strBaseDir  = "/drv3/hm3/Data/IMDB/aclImdb"
base_dir = pathlib.Path( strBaseDir )
train_ds =  keras.utils.text_dataset_from_directory( base_dir / "train", batch_size=batch_size )
val_ds = keras.utils.text_dataset_from_directory( base_dir / "val", batch_size=batch_size )
test_ds = keras.utils.text_dataset_from_directory( base_dir / "test", batch_size=batch_size )

# verify by printing that the 3 data sets were created correctly
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

text_only_train_ds = train_ds.map(lambda x, y: x)



Found 20000 files belonging to 2 classes.


I0000 00:00:1743194416.694611   33122 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9164 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:08:00.0, compute capability: 8.6


Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
inputs.shape: (12,)
inputs.dtype: <dtype: 'string'>
targets.shape: (12,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'I give this film it\'s props that it is very well made and reasonably well acted. BUt I couldn\'t get past the implausibility of the whole thing.<br /><br />First and foremost, a game built around the notion of "Russian Roulette" that has to fill on hour. the big problem is that if you are doing a "live" show, you run the possibility that your first contestant will be the one unlucky enough to draw the "real" bullet. Then what do you do? You have 50 minutes of show to fill and nothing to show. The corollary is that Okay, you get to the end and the first five contestants survive, which means number six has the bullet and can\'t possibly get the payout. He isn\'t going to shoot himself at that point, so it\'s kind of anti-climatic. <br /><br />second problem, almost as big. Human 

In [4]:
""" 
The code in the cell above does what the code in this cell is supposnote:

    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)
"""

' \nThe code in the cell above does what the code in this cell is supposed to do\n\nbatch_size = 32\nbase_dir = pathlib.Path("aclImdb")\nval_dir = base_dir / "val"\ntrain_dir = base_dir / "train"\nfor category in ("neg", "pos"):\n    os.makedirs(val_dir / category)\n    files = os.listdir(train_dir / category)\n    random.Random(1337).shuffle(files)\n    num_val_samples = int(0.2 * len(files))\n    val_files = files[-num_val_samples:]\n    for fname in val_files:\n        shutil.move(train_dir / category / fname,\n                    val_dir / category / fname)\n\ntrain_ds = keras.utils.text_dataset_from_directory(\n    "aclImdb/train", batch_size=batch_size\n)\nval_ds = keras.utils.text_dataset_from_directory(\n    "aclImdb/val", batch_size=batch_size\n)\ntest_ds = keras.utils.text_dataset_from_directory(\n    "aclImdb/test", batch_size=batch_size\n)\ntext_only_train_ds = train_ds.map(lambda x, y: x)\n'

In [3]:
## Prepare integer sequence datasets

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

# this code emits a message at the end:
# 2025-03-13 17:25:48.190162: I tensorflow/core/framework/local_rendezvous.cc:405]
# Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
# message could be just a diagnostic warning  

2025-03-28 16:40:34.904387: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# code in cell is written to test the tf.one_hot error in teh cell that follows

# Assuming 'inputs' is some tensor that you want to one hot encode.
inputs = tf.constant([1.0, 2.0, 3.0]) #example of wrong type.
inputs = tf.cast(inputs, dtype=tf.int32) #correcting the type.
max_tokens = 5
embedded = tf.one_hot(inputs, depth=max_tokens)
print(embedded)


tf.Tensor(
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]], shape=(3, 5), dtype=float32)


In [5]:
# create a simple model. Note that the model *must* use immediately after the input
# an embedding layer. Since I did not specify the type of nGram during the call to TextVectorization
# ngrams = 1 (default value )


max_tokens = 1000  # Example vocabulary size
embedding_dim = 128 #Example embedding dimension

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=embedding_dim)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 128)      │       128,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 169,281 (661.25 KB)

 Trainable params: 169,281 (661.25 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10


I0000 00:00:1743172268.618720   15771 cuda_dnn.cc:529] Loaded cuDNN version 90501


1667/1667 ━━━━━━━━━━━━━━━━━━━━ 76s 44ms/step - accuracy: 0.6447 - loss: 0.6107 - val_accuracy: 0.7542 - val_loss: 0.4981
Epoch 2/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 73s 44ms/step - accuracy: 0.8019 - loss: 0.4632 - val_accuracy: 0.8368 - val_loss: 0.3929
Epoch 3/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8245 - loss: 0.4251 - val_accuracy: 0.8420 - val_loss: 0.3795
Epoch 4/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8304 - loss: 0.4068 - val_accuracy: 0.8026 - val_loss: 0.4211
Epoch 5/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8440 - loss: 0.3790 - val_accuracy: 0.8612 - val_loss: 0.3464
Epoch 6/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8569 - loss: 0.3618 - val_accuracy: 0.8554 - val_loss: 0.3521
Epoch 7/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 73s 44ms/step - accuracy: 0.8673 - loss: 0.3418 - val_accuracy: 0.8580 - val_loss: 0.3276
Epoch 8/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 72s 43ms/step - accuracy: 0.8709 - loss: 0.33

In [9]:
## Adding an embedding layer imediately after the input layer 

embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

In [10]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, None, 256)      │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        73,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 330,049 (1.26 MB)

 Trainable params: 330,049 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 75s 44ms/step - accuracy: 0.6492 - loss: 0.6198 - val_accuracy: 0.8220 - val_loss: 0.4237
Epoch 2/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.7995 - loss: 0.4729 - val_accuracy: 0.8446 - val_loss: 0.3853
Epoch 3/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 73s 44ms/step - accuracy: 0.8205 - loss: 0.4222 - val_accuracy: 0.8350 - val_loss: 0.4012
Epoch 4/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 72s 43ms/step - accuracy: 0.8432 - loss: 0.3907 - val_accuracy: 0.8654 - val_loss: 0.3298
Epoch 5/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8531 - loss: 0.3599 - val_accuracy: 0.8664 - val_loss: 0.3306
Epoch 6/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8643 - loss: 0.3423 - val_accuracy: 0.8464 - val_loss: 0.3328
Epoch 7/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8713 - loss: 0.3230 - val_accuracy: 0.8416 - val_loss: 0.3470
Epoch 8/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.8735 -

In [11]:
## Using an Embedding  layer with masking enabled 

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 256) │    256,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input_layer_2[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 64)        │     73,984 │ embedding_3[0][0… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 330,049 (1.26 MB)

 Trainable params: 330,049 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 66s 39ms/step - accuracy: 0.6862 - loss: 0.5669 - val_accuracy: 0.8216 - val_loss: 0.4018
Epoch 2/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 66s 39ms/step - accuracy: 0.8244 - loss: 0.4129 - val_accuracy: 0.8368 - val_loss: 0.3706
Epoch 3/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 66s 40ms/step - accuracy: 0.8424 - loss: 0.3692 - val_accuracy: 0.8578 - val_loss: 0.3284
Epoch 4/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 66s 40ms/step - accuracy: 0.8571 - loss: 0.3450 - val_accuracy: 0.8602 - val_loss: 0.3225
Epoch 5/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 67s 40ms/step - accuracy: 0.8693 - loss: 0.3239 - val_accuracy: 0.8654 - val_loss: 0.3155
Epoch 6/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 67s 40ms/step - accuracy: 0.8745 - loss: 0.3049 - val_accuracy: 0.8624 - val_loss: 0.3216
Epoch 7/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 67s 40ms/step - accuracy: 0.8819 - loss: 0.2873 - val_accuracy: 0.8678 - val_loss: 0.3122
Epoch 8/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 67s 40ms/step - accuracy: 0.8907 -

In [12]:
path_to_glove_file = "/drv3/hm3/code/python/tf2.18/tf2.18/local/TxtProc/glove.6B/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [14]:
embedding_dim = 100
max_tokens= 20000

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

        

In [15]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

In [16]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 100) │  2,000,000 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None)      │          0 │ input_layer_3[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 64)        │     34,048 │ embedding_4[0][0… │
│ (Bidirectional)     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64)        │          0 │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         65 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,034,113 (7.76 MB)

 Trainable params: 34,113 (133.25 KB)

 Non-trainable params: 2,000,000 (7.63 MB)

Epoch 1/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 68s 40ms/step - accuracy: 0.6419 - loss: 0.6169 - val_accuracy: 0.8026 - val_loss: 0.4190
Epoch 2/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 67s 40ms/step - accuracy: 0.8075 - loss: 0.4325 - val_accuracy: 0.8388 - val_loss: 0.3678
Epoch 3/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 66s 40ms/step - accuracy: 0.8383 - loss: 0.3756 - val_accuracy: 0.8622 - val_loss: 0.3253
Epoch 4/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 65s 39ms/step - accuracy: 0.8549 - loss: 0.3433 - val_accuracy: 0.8734 - val_loss: 0.3080
Epoch 5/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 64s 38ms/step - accuracy: 0.8682 - loss: 0.3161 - val_accuracy: 0.8702 - val_loss: 0.3166
Epoch 6/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 66s 40ms/step - accuracy: 0.8826 - loss: 0.2932 - val_accuracy: 0.8746 - val_loss: 0.2971
Epoch 7/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 65s 39ms/step - accuracy: 0.8902 - loss: 0.2735 - val_accuracy: 0.8748 - val_loss: 0.3089
Epoch 8/10
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 65s 39ms/step - accuracy: 0.8968 -